In [ ]:
import pandas as pd

# 正しいエンコーディングで読み込み
df = pd.read_csv('../csv/yahoo_sale_all_data.csv', encoding='shift_jis')

# 必要な列だけ抽出
columns_to_keep = [
    "OrderTime",
    "SubCode",
    "Title",
    "UnitPrice",
    "TotalMallCouponDiscount",
    "UsePoint",
    "ShipCharge"
]

df_filtered = df[columns_to_keep]

# df_filtered.to_csv('../csv/yahoo_sale_all_data_filtered.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd

# ファイル読み込み
df = pd.read_csv("../csv/yahoo_sale_all_data_filtered.csv", encoding="utf-8-sig")

# 展開済みデータ保存リスト
all_rows = []

for _, row in df.iterrows():
    subcode_raw = str(row["SubCode"])
    title_raw = str(row["Title"])
    price_raw = str(row["UnitPrice"])

    # 複数商品購入かどうか
    is_multi = "&L" in subcode_raw or "&L" in title_raw or "&L" in price_raw

    # 複数商品の場合
    if is_multi:
        subcodes = subcode_raw.lstrip('&').split('&L')
        titles = title_raw.lstrip('&').split('&L')
        prices = price_raw.lstrip('&').split('&L')

        for sc, ti, pr in zip(subcodes, titles, prices):
            subcode_id, subcode_val = sc.split('=', 1) if '=' in sc else ('', sc)
            title_id, title_val = ti.split('=', 1) if '=' in ti else ('', ti)
            price_id, price_val = pr.split('=', 1) if '=' in pr else ('', pr)

            if subcode_id == title_id == price_id:
                new_row = row.copy()
                new_row["SubCode"] = subcode_val.strip()
                new_row["Title"] = title_val.strip()
                new_row["UnitPrice"] = int(price_val) if price_val.strip().isdigit() else None
                all_rows.append(new_row)

    # L1=だけ1商品で入っている場合（削除して出力）
    elif "L1=" in title_raw and "&L" not in title_raw:
        new_row = row.copy()
        new_row["SubCode"] = subcode_raw.replace("L1=", "").strip()
        new_row["Title"] = title_raw.replace("L1=", "").strip()
        new_row["UnitPrice"] = int(price_raw.replace("L1=", "").strip()) if price_raw.replace("L1=", "").strip().isdigit() else None
        all_rows.append(new_row)

    # 通常行はそのまま
    else:
        all_rows.append(row)

# DataFrameとして保存
df_final = pd.DataFrame(all_rows)

# CSVに保存
df_final.to_csv("yahoo_sale_all_data_expanded_all_rows.csv", index=False, encoding="utf-8-sig")

print("✅ 完了：正常データも含めて、複数商品データを分割展開して保存しました！")

✅ 完了：正常データも含めて、複数商品データを分割展開して保存しました！


In [23]:
import pandas as pd

# データ読み込み
df = pd.read_csv("yahoo_sale_all_data_expanded_all_rows.csv", encoding="utf-8-sig")
df_brands = pd.read_csv("../csv/brand_list_separated.csv", encoding="utf-8-sig")  # 列名: ブランド名（英語）, ブランド名（日本語）

# ブランド辞書（長い順で優先マッチ）
brand_list = sorted(set(df_brands['ブランド名（日本語）'].dropna().tolist() + df_brands['ブランド名（英語）'].dropna().tolist()), key=len, reverse=True)

# 季節ラベル抽出関数
def label_season(text):
    if pd.isna(text):
        return ''
    text = str(text).lower()
    if any(kw in text for kw in ['春 夏 秋 冬', 'オールシーズン']):
        return '通年'
    elif any(kw in text for kw in ['秋冬', "秋 冬"]):
        return '秋冬'
    elif any(kw in text for kw in ['春秋', '春 秋']):
        return '春秋'
    elif any(kw in text for kw in ['春夏', "春 夏"]):
        return '春夏'
    return ''

# カテゴリラベル抽出関数
def label_category(text):
    if pd.isna(text):
        return ''
    text = str(text).lower()
    category_keywords = {
        'Outer': ['コート', 'ジャケット', 'ブルゾン', 'アウター', 'ダウン', 'パーカー'],
        'Suit': ['スーツ', 'セットアップ', 'ジャケット＆パンツ', '上下セット'],
        'Tops': ['トップス', 'ブラウス', 'シャツ', 'ニット', 'カーディガン', 'カットソー', 'セーター', 'tシャツ'],
        'Bottoms': ['パンツ', 'スカート', 'ボトムス', 'ショートパンツ', 'ガウチョ'],
        'Dress': ['ワンピース', 'ドレス'],
        'Accessory': ['バッグ', '帽子', 'ハット', 'ベルト', 'スヌード', 'マフラー', 'ストール', '手袋', 'アクセサリー']
    }
    for category, keywords in category_keywords.items():
        if any(kw in text for kw in keywords):
            return category
    return ''

# ラベルを適用
df['季節ラベル'] = df['Title'].apply(label_season)
df['カテゴリラベル'] = df['Title'].apply(label_category)

# 結果保存
df.to_csv("../csv/final_ラベル付き.csv", index=False, encoding='utf-8-sig')


In [26]:
import pandas as pd
import re

# データ読み込み
df = pd.read_csv("yahoo_sale_all_data_expanded_all_rows.csv", encoding="utf-8-sig")

# 分割対象の記号と空白で Title を分割
# → 空白, /, ☆, *, ♪ を正規表現で分割トリガーに
split_pattern = r'[ /☆*♪]+'

# 分割処理（NaN対策しつつ）
title_split_df = df["Title"].fillna('').apply(lambda x: pd.Series(re.split(split_pattern, x.strip())))

# カラム名を自動生成
title_split_df.columns = [f"Title_{i+1}" for i in range(title_split_df.shape[1])]

# 結合
df_combined = pd.concat([df, title_split_df], axis=1)

# 保存
df_combined.to_csv("yahoo_title_split_full.csv", index=False, encoding="utf-8-sig")

print("✅ 空白・/・☆・*・♪ で分割し、保存しました！")


✅ 空白・/・☆・*・♪ で分割し、保存しました！


In [34]:
# データ読み込み
df = pd.read_csv("yahoo_title_split_full.csv", encoding="utf-8-sig")

df = df["Title_2"]

df.to_csv("yahoo_title_split_2.csv", index=False, encoding="utf-8-sig")
print("✅ Title_2 のみを抽出して保存しました！")

✅ Title_2 のみを抽出して保存しました！


In [36]:
import pandas as pd

# CSV読み込み
df_brands = pd.read_csv("yahoo_title_split_2.csv", encoding="utf-8-sig")
df_final = pd.read_csv("../csv/final_ラベル付き.csv", encoding="utf-8-sig")

# 横方向（列方向）に結合
df_final = pd.concat([df_final, df_brands], axis=1)
df_final.to_csv("../dataset/final_yahoo_20240422-2025-0412.csv", index=False, encoding="utf-8-sig")

In [3]:
import pandas as pd

df = pd.read_csv("../dataset/all_sales_data_2021_2025_labeled copy.csv", encoding="utf-8-sig")

df["Brand_Japanese"].unique()

array(['ビースリー', 'ユキトリイ', 'バーバリー', 'ピサーノ', 'フラジール', 'レリアン', 'エプリス',
       'マーガレットハウエル', 'イエーガー', 'アナイ', 'ラルフ ローレン', 'レオナール', 'ベルソフィア',
       'コージワタナベ', 'スキャパ', 'ビープラス', 'グレープ元町', 'ジブリーヌ', 'ランズエンド', 'レキップ',
       '詩仙堂', 'セレブ・モイ', '23区', 'オッピオ', 'グリーンレーベルリラクシング', 'デリススポーツ',
       'ボルボネーゼ', 'イエナ', 'マークジェイコブス', 'ニューヨーカー', 'ジェラールダレル', 'トクコプルミエヴォル',
       'ノーブランド', 'ヴァンドゥーオクトーブル', '自由区', 'イヴサンローラン', 'サンファン', 'ソワールドルチェ',
       'ドゥクラッセ', 'ヒトシタムラ', 'ユナイテッドアローズ', '三年坂', 'イギン', 'インディヴィ',
       'アクアスキュータム', 'コンセプト', 'ソプラニードンナ', 'メーカーズシャツ鎌倉', 'アマンジャッキー', 'ウンガロ',
       'エヴェックスバイクリツィア', 'ファースアファース', 'マニ', 'マリナリナルディ', 'ランセル', 'リファンネ',
       'レマロン', 'ロペ', 'グアルディ', 'ソニアリキエル', 'ヨウジヤマモト', 'ロブジェ', 'アンタイトル',
       'エフデ', 'エマジェイムス', 'クリスチャンディオール', 'クードシャンス', 'ココ', 'トピィーズ',
       'ナラカミーチェ', '東京イギン', '組曲', 'エディットコロン', 'エルミダ', 'ローラアシュレイ',
       'セオリーリュクス', 'トランスワーク', 'ジャンフランコフェレ', 'タルボット', 'ボディドレッシング デラックス',
       'エレガンス', 'ボンシリエ', 'アイグナー', 'ジバンシイ', 'ジュンコシマダ', 'マッキントッシュ', 'サクラ',
       'ボッシュ', 'バレンザ